In [27]:

import numpy as np
import pandas as pd

inputs_other = np.load('inputs_others_train.npy')
yield_train = np.load('yield_train.npy')
inputs_weather_train = np.load('inputs_weather_train.npy')
clusterID_genotype = np.load('clusterID_genotype.npy')

In [2]:
#jsut copy of imporrt use over multible file come back and clean this up
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import logistic
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler

C:\Users\Austin\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
#compress weather
def buildWeather(inputs_weather_train):
    new_weather = np.column_stack((np.repeat(np.arange(93028),214),inputs_weather_train.reshape(93028*214,-1)))
    new_weather = pd.DataFrame(new_weather, columns = ['PR','ADNI','AP','ARH','MDNI','MaxSur','MinSur','AvgSur'])
    return new_weather

In [4]:
#pulls one day out of weather file
def pull_one_day(dataWeather,day,ran):
    dayOne = []
    #dfw = pd.DataFrame(columns = ['AP','ARH','MDNI','MxSur','MnSur','MnSur','AgSur'])
    for i in range(ran):
        dayOne.append(dataWeather[i][day].tolist())
    #dfw.append(pd.DataFrame((dataWeather[i][0].tolist()) ,columns = ['AP','ARH','MDNI','MxSur','MnSur','MnSur','AgSur']))
    dfw = pd.DataFrame(dayOne,columns = ['ADNI','AP','ARH','MDNI','MaxSur','MinSur','AvgSur'])
    return dfw

In [5]:
#compress other data and yeid
def merge_yield_other(inputs_other,yield_train):
    inputs_other_df = pd.DataFrame(inputs_other,columns = ['MG','GID','State','Year','loc'])
    yield_train_df = pd.DataFrame(yield_train,columns = ['yield'])
    inputs_other_df.insert(5,"yield", yield_train_df['yield'])
    return inputs_other_df

In [6]:
def make_data_small(dataFrame,size,random):
    if random:
        return dataFrame.sample(size)
    else:
        return dataFrame.head(size)
    

In [7]:
def buildWeather(inputs_weather_train):
    new_weather = np.column_stack((np.repeat(np.arange(93028),214),inputs_weather_train.reshape(93028*214,-1)))
    new_weather = pd.DataFrame(new_weather, columns = ['PR','ADNI','AP','ARH','MDNI','MaxSur','MinSur','AvgSur'])
    return new_weather

In [8]:
def group_weather_yeild(weather,yields):
    return pd.concat([yields,weather], axis=1)
    

In [10]:
def DecsisionTree(data,day):
    data_set = data
    data_set.head(12)
    y = data_set['yield']
    x = data_set['AvgSur']
    
    y = (y.to_numpy()).reshape(-1, 1)
    x = (x.to_numpy()).reshape(-1, 1)
    #['ADNI','AP','ARH','MDNI','MaxSur','MinSur',]
    test_size = 0.5
    seed = 5
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size, random_state = seed)
    
    
    model = DecisionTreeRegressor()
    model.fit(x_train, y_train)
    predictions = model.predict(x_train)
    predictions = model.predict(x_test)
    x_train = pd.DataFrame(x_train)
    y_train = pd.DataFrame(y_train)
    
    x_test = pd.DataFrame(x_test)
    y_test = pd.DataFrame(y_test)
      
    test_other = np.load('inputs_others_test.npy')
    inputs_weather_test = np.load('inputs_weather_test.npy')

    sample_two = pull_one_day(inputs_weather_test,day,10337)

    sample_two = pd.DataFrame(sample_two)
    test_other = pd.DataFrame(test_other)

    test_data = group_weather_yeild(sample_two,test_other)
    test_data = test_data['AvgSur']
    test_data = (test_data.to_numpy()).reshape(-1, 1)
    predictions = model.predict(test_data)
    
    return predictions

In [50]:
def model_not_lstm(data):
    
    data_set = data
    
    data_set.head(12)
    
    y = data_set['yield']

    x = data_set['AvgSur']
    #['ADNI','AP','ARH','MDNI','MaxSur','MinSur',]
    x.head()
    y.head()
    test_size = 0.5
    seed = 5
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size, random_state = seed)
    #model
    model = DecisionTreeClassifier()
    
    model.fit(x_train, y_train)
    
    predictions = model.predict(x_train)
    print(accuracy_score(y_train, predictions))
    predictions = model.predict(x_test)
    print(accuracy_score(y_test, predictions))   
    
    df = x_test.copy()
    df['Actual'] = y_test
    df['Prediction'] = predictions
    df

In [54]:
def lstm_model(data):
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data)
    data_set = data
    data_set.head(12)
    y = data_set['yield'].reshape(-1, 1)
    x = data_set[['ADNI','AP','ARH','MDNI','MaxSur','MinSur','AvgSur']].reshape(-1, 1)
    x.head()
    y.head()
    
    test_size = 0.5
    
    seed = 5
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size, random_state = seed)
    
    x_train = x_train/255.0
    x_test = x_test/255.0

    model = Sequential()

    model.add(LSTM(128,input_shape=(x_train.shape[1:]),activation ='relu',return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(128,activation ='relu'))
    model.add(Dropout(0.2))

    model.add(LSTM(32,activation ='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(10,activation='softmax'))

    opt = tf.keras.optimizers.Adam(Lr=1e-3,decay=1e-5)

    model.compile(Loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

    model.fit(x_train,y_tran,epochs=2,validation_data=(x_test,y_test))
    

In [12]:
#sample_one = pull_one_day(inputs_weather_train,0,93028)
#yield_merge = merge_yield_other(inputs_other,yield_train)
#group_data = group_weather_yeild(sample_one,yield_merge)
#group_data.head(15)
#group_data.tail(15)
#group_data.describe()
#sample_one.describe()
#model_not_lstm(group_data)
#lstm_model(group_data)

prediction_each_day = []
for i in range(213):
    sample_one = pull_one_day(inputs_weather_train,i,93028)
    yield_merge = merge_yield_other(inputs_other,yield_train)
    group_data = group_weather_yeild(sample_one,yield_merge)
    prediction_each_day.append(DecsisionTree(group_data,i))
    

    
print(prediction_each_day[0])
#predictions = DecsisionTree(group_data)

#predict.head(15)
#predict.shape
#predictions = pd.DataFrame(predictions)
#predictions.head(15)
#predictions.shape
#np.save("predictions",(predictions.to_numpy()))

[45.57931034 48.15347594 58.50487805 ... 53.34965035 53.29395973
 61.0675    ]


In [15]:
tmp = pd.DataFrame(prediction_each_day)
tmp.shape

(213, 10337)

In [16]:
tmp.head(15)

,0,1,2,3,4,5,6,7,8,9,...,10327,10328,10329,10330,10331,10332,10333,10334,10335,10336
0,45.579310,48.153476,58.504878,48.836905,61.307018,26.041935,53.000000,59.968919,58.788525,55.904478,...,52.537288,27.269388,23.104545,48.468103,60.413333,55.383019,65.559420,53.349650,53.293960,61.0675
1,45.579310,48.328667,58.504878,52.378512,59.557480,25.356818,51.424096,59.968919,58.788525,59.519048,...,52.537288,27.269388,23.104545,48.468103,60.413333,53.588889,65.559420,53.349650,53.293960,61.0675
2,45.579310,48.328667,58.504878,48.836905,61.307018,26.041935,51.424096,59.968919,58.788525,32.960504,...,52.537288,32.960504,23.104545,48.468103,60.413333,53.588889,65.559420,53.349650,53.293960,61.0675
3,45.579310,48.328667,58.203409,48.836905,61.307018,26.041935,44.760326,60.827500,58.788525,59.519048,...,52.537288,37.128646,23.104545,48.468103,60.413333,53.588889,65.559420,53.349650,53.293960,61.0675
4,57.523636,48.328667,58.504878,48.836905,61.307018,26.041935,51.424096,51.737912,58.788525,59.519048,...,52.537288,26.835200,23.104545,48.468103,60.413333,53.588889,65.559420,53.349650,56.183333,61.0675
5,45.579310,48.328667,58.504878,47.110435,60.672794,26.041935,51.424096,59.968919,57.841875,59.519048,...,52.537288,27.269388,23.104545,48.468103,60.413333,53.588889,65.559420,53.349650,53.293960,61.0675
6,45.579310,48.328667,58.504878,47.899242,61.307018,26.041935,51.424096,59.968919,58.788525,59.519048,...,52.537288,27.269388,23.104545,48.468103,60.413333,54.508000,65.559420,56.102941,53.293960,61.0675
7,57.484444,48.498125,58.504878,48.836905,61.307018,26.041935,51.424096,63.607960,58.788525,59.519048,...,52.537288,34.298013,23.104545,48.468103,60.413333,53.588889,65.559420,53.349650,53.293960,61.0675
8,45.579310,48.328667,66.487387,48.798450,61.307018,26.041935,44.890000,59.968919,58.788525,46.260825,...,52.537288,27.269388,55.161017,48.468103,65.964000,53.588889,61.346825,53.349650,53.293960,61.0675
9,45.579310,48.328667,55.147917,48.836905,61.307018,26.041935,51.424096,60.398214,48.871154,59.519048,...,50.775893,27.269388,23.104545,50.719380,60.413333,53.588889,60.906838,53.349650,52.825000,61.0675


In [17]:
tmp[0].mean()

46.47413929350385

In [22]:
prediction_over_time=[]
for i in range(10337):
    prediction_over_time.append(tmp[i].mean())

In [24]:
prediction_over_time = pd.DataFrame(prediction_over_time)
prediction_over_time.head(15)
prediction_over_time.shape

(10337, 1)

In [26]:
np.save("prediction_over_time",(prediction_over_time.to_numpy()))